# CSE688: Practical and Application of Machine Learning - Spring 2021
## Assignment 3-2
### Authors

- B073040018 朱劭璿
- B072010029 陳居廷

### References
https://www.tensorflow.org/tutorials/text/classify_text_with_bert

#### Sentiment analysis on the sentences in the attachment `hw3-2-dataset`. Classify the sentences into two categories: possitive (1) amd negative (0)

In [1]:
import tensorflow as tf
import numpy as np

DATA_DIR = '../hw3-2-dataset/'
train_label = []
train_data = []
with open(DATA_DIR+'training.txt', 'r') as f:
    lines = f.readlines()
for l in lines:
    l = l[:-1].split('\t')
    train_label.append(int(l[0]))
    train_data.append(l[1])
    
train_label = np.array(train_label)
train_data = np.array(train_data)

shuffler = np.random.permutation(train_label.shape[0])
train_label = train_label[shuffler]
train_data = train_data[shuffler]

val_split = int(0.3 * train_label.shape[0])

val_label = train_label[:val_split]
val_data = train_data[:val_split]
train_label = train_label[val_split:]
train_data = train_data[val_split:]

ds_train = tf.data.Dataset.from_tensor_slices((train_data, train_label))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(train_label.shape[0])
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_val = tf.data.Dataset.from_tensor_slices((val_data, val_label))
ds_val = ds_val.batch(32)
ds_val = ds_val.cache()
ds_tval= ds_val.prefetch(tf.data.experimental.AUTOTUNE)

print('Label:', train_label[0])
print('Sentence:', train_data[0])

Label: 1
Sentence: I liked Crash a lot, I liked Brokeback Mountain, but GN & GL was the important movie that the other two were hyped as being...


In [2]:
import tensorflow_hub as hub
import tensorflow_text as text

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

classifier_model = build_classifier_model()
classifier_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'pooled_output': (N 109482241   preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
_______________________________________________________________________________________

In [4]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

init_lr = 3e-5
optimizer = tf.keras.optimizers.Adam(init_lr)

classifier_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [5]:
epochs = 2
history = classifier_model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=epochs
)

Epoch 1/2
156/156 [==============================] - 91s 585ms/step - loss: 0.0687 - binary_accuracy: 0.9724 - val_loss: 0.0086 - val_binary_accuracy: 0.9967
Epoch 2/2
156/156 [==============================] - 90s 578ms/step - loss: 0.0073 - binary_accuracy: 0.9978 - val_loss: 0.0049 - val_binary_accuracy: 0.9981
